In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import pandas as pd


In [ ]:
def load_config(config_num):
    img_file = f'heatmaps/move_heatmap_config{config_num}.jpg'
    xml_file = f'xmls/move_heatmap_config{config_num}.xml'

    img = cv2.imread(img_file)
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    tree = ET.parse(xml_file)
    root = tree.getroot()

    xmins = [int(xmin.text) for xmin in root.findall('object/bndbox/xmin')]
    ymins = [int(ymin.text) for ymin in root.findall('object/bndbox/ymin')]
    xmaxs = [int(xmax.text) for xmax in root.findall('object/bndbox/xmax')]
    ymaxs = [int(ymax.text) for ymax in root.findall('object/bndbox/ymax')]

    height = int(root.find('size/height').text)
    width = int(root.find('size/width').text)

    xml_data = open(xml_file).read()
    num_obj = xml_data.count('</object>')

    return img_gray, num_obj, xmins, ymins, xmaxs, ymaxs, height, width


In [ ]:
tree = ET.parse('xmls/move_heatmap_config0.xml')
root = tree.getroot()

names = [element.text for element in root.findall('object/name')]
print(names)


In [ ]:
all_scores = []

configs = range(0, 7)

for config in configs:
    img_gray, num_obj, xmins, ymins, xmaxs, ymaxs, height, width = load_config(config)
    scores = []
    for i in range(0, num_obj):
        mask = np.zeros([height, width])

        xmin, ymin, xmax, ymax = xmins[i], ymins[i], xmaxs[i], ymaxs[i]

        bbox = np.array([xmin, ymin, xmax, ymax])

        mask[bbox[1]:bbox[3], bbox[0]:bbox[2]] = 1

        object = np.multiply(mask, img_gray)

        score = np.amax(object) / np.amax(img_gray)
        scores.append(score)

    all_scores.append(scores)

all_scores_df = pd.DataFrame(all_scores, columns=names)

all_scores_df.to_csv('heatmap_analysis_results.csv', index=False)
all_scores_df

In [ ]:
max_score = all_scores_df.iloc[:, -1].max()

cmap = plt.cm.get_cmap('Blues')

for i in range(len(all_scores_df)):
    color = cmap((all_scores_df.iloc[i, -1] / max_score) ** 10)
    plt.bar(i, all_scores_df.iloc[i, -1], color=color, width=0.5)

plt.xlabel('Configuration No.')
plt.ylabel('Attention Score')
plt.title('Attention Score of the Checkout Button')
plt.rcParams["figure.figsize"] = (6, 3)

plt.show()